-----

## Batcher Even-Odd sort

Burton Rosenberg

14 June 2023


----

In a 1968 report, Ken Batcher presented two ways to sort that can take advantage of multiple data pathways and multiple computing units. The diagrams for these networks suggest actual physical pathways connecting data comparison units. The data flows from left to right along the pathways, entering the units to be compared, swapped if necessary, and continue on exiting the units.

As the data flows, units become dependent on other units, and one arranges the units in layers to clarify the stages of processing. The number of the layers is the processing time for the network. The networks invented by Batcher for $n$ numbers with have $\log^2 n$ layers. These networks are therefore faster ways to sort numbers.

To familiarize outselves with the diagram, here is how the Even-Odd merging network would sort 4 numbers,

<pre>
a ---+----------+------- s = max(a,b,c,d)
     |          |
b ---+-- x  y --+--+---  t
                   |
c ---+-- y  x --+--+---  u
     |          |
d ---+----------+------- v = min(a,b,c,d)
</pre>

The crossbar widgets swap or not the two elements entering on the left to leave ordered to the right. The circuit works by first establishing two ordered sublist (first layer) and then merging them (second and third layer). While the exact pattern is not evident in such a small example, in this small example the two sublists are split and the larger are compared to the larger, and the smaller to the smaller (second layer). The largest and smallest are now certainly in the correct place. Then the middle two are considered (third layer).

##### The 0-1 Principle 

Batcher did not know this, but it is a fact that if a sorting network works when the values to sort are restricted to $\{\,0,1\,\}$, then it works in general.

So our inputs, as a list or vector will always be of the form $i$ 1's followed by $n-i$ zeros, where $n$ is a pure power of two. This will be denoted as $\langle\,i\,\rangle_n$, 

##### The Induction Hypothesis and Goal

We assume to  have $\langle\,i\,\rangle_n$ and $\langle\,j\,\rangle_n$ and want their merge, $\langle\,i+j\,\rangle_{2n}$.

##### Even-Odd Split

The list $\langle\,i\,\rangle_n$ is split into even and odd locations. If $i$ is even, then the two splits lists are both $\langle\,i/2\,\rangle_{n/2}$. If $i$ is odd the lists are $\langle\,(i+1)/2\,\rangle_{n/2}$ and $\langle\,(i-1)/2\,\rangle_{n/2}$. The same considerations for the list $\langle\,j\,\rangle_n$. 

In the case of $i$ odd, the list with the greater number of 1's is call the top list, and the other the bottom. For $i$ even either one is called the top and the other is called the bottom.

##### Recursion 

Merge the tops lists together; merge the bottom lists together. This is a recursive structure since these are lists of length $n/2$.

There are three possible outcomes for the two merged lists.

- Both are $\langle\,(i+j)/2\,\rangle_{n}$.
- One is $\langle\,(i+j+1)/2\,\rangle_{n}$ and the other $\langle\,(i+j-1)/2\,\rangle_{n}$.
- One is $\langle\,(i+j)/2+1\,\rangle_{n}$ and the other $\langle\,(i+j)/2-1\,\rangle_{n}$.

##### Combine

If the highest element of the top list is zero, the both lists are completely zero, and the combine step succeeds. 

Consider the case that the highest elment of the top list is one. The highest element of the top list is set aside and the lowest element of the bottom list is set aside. The two lists, reduced to the remaining $n-1$ elements, are paired element-wise. According to the three cases in the recursion step, the cases are, 

- The list $\langle\,(i+j)/2-1\,\rangle_{n-1}$ is paired with $\langle\,(i+j)/2\,\rangle_{n-1}$
- Both lists in the pair are $\langle\,(i+j-1)/2\,\rangle_{n-1}$.
- The list $\langle\,(i+j)/2\,\rangle_{n-1}$ is paired with $\langle\,(i+j)/2-1\,\rangle_{n-1}$.

So all but one pairing will be 0-0 or 1-1. The combined list is sorted by the swaping of the (possible) 0-1 pairing.

##### Overall algorithm and analysis

The overall network combines two logarithmically scalling intentions. The input will be divided into 2, 4, 8, and so on, consecutive wires, and merged. 

For 2, the two single wires are trivially sorted, and the output is sorted; and is simply the swap device between these wires.

Then the pairs of sorted 2's are merged into 4's, etc. With $\log n$ levels of merge.

The above describes the Batcher even-odd merge, which breaks the task of merging to $k$ length lists to two size $k/2$ merges. This is solved recursively. In terms of what circuits are generated at each recursion level, it is a single bank of swap devices. Hence the complete result from the $\log n$ depth recursion of subproblems is $\log n$ layers of swap devices.

Hence the overall depth of $(\log n)^2$.

### Python code

Follows it python code carrying out the sort; however this code does not attempt to simulate a network.



In [1]:
# batcher's even odd sort

class BatcherOddEvenMerge:
    
    def __init__(self):
        pass
    
    def sort_aux(self,a,b):
        
        if len(a)==1:
            return [min(a[0],b[0]),max(a[0],b[0])]

        # using batcher's numbering for odd and even 
        # (contrary to based at zero arrays)
        a_odd = a[0::2]
        a_even = a[1::2]
        b_odd = b[0::2]
        b_even = b[1::2]

        odd = self.sort_aux(a_odd,b_odd)
        even = self.sort_aux(a_even,b_even)

        c = [0]*(len(a)+len(b))
        c[0] = odd[0]
        c[-1] = even[-1]
        c[1:-1:2] = odd[1::]
        c[2:-1:2] = even[0:len(even)-1]
        for i in range(1,len(c)-1,2):
            if c[i]>c[i+1] : c[i],c[i+1]=c[i+1],c[i]
        return c
    
    def sort(self,a,b):
        
        def power_of_two(n):
            while n>1:
                if n%2==1:
                    return False
                n //= 2
            return True
        
        assert len(a)==len(b), 'lists must be of equal size'
        assert power_of_two(len(a)), 'list length must be a power of 2'
        return self.sort_aux(a,b)

In [2]:
import random

k= 16
a = [i for i in range(k)]
#random.shuffle(a)
print(a)
b = [i for i in range(k)]
#random.shuffle(b)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [3]:
bod = BatcherOddEvenMerge()
bod.sort(a,b)

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15]

In [4]:
a = [0,0,0,0,1,1,1,1]
b = [0,1,1,1,1,1,1,1]
bod.sort(a,b)


[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]